## 고속 옵티마이저

표준적인 경사하강법 대신 더 빠른 옵티마이저를 사용하는 것은 굉장히 좋은 선택입니다.

### momentum optimization

In [190]:
import keras
from zipfile import ZipFile
import pandas as pd
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as f:
    data=io.BytesIO(f.read("고객 대출등급 분류 해커톤/train.csv"))
from sklearn import model_selection as mod
import numpy as np
import tensorflow as tf

data=pd.read_csv(data)

표준적인 경사하강법은 다음과 같은 방법임을 우리는 알고있습니다.

$\theta$ -> $\theta - \eta \triangledown_{\theta} J(\theta)$

여기서 $J(\theta)$ 는 $\theta$에 따른 손실(자코비안)행렬(gradient)입니다.

모멘텀(momentum)은 관성이라는 관점에서 이 경사도를 이해합니다.

$m$ -> $\beta m- \eta \triangledown_{\theta} J(\theta)$

$\theta$ -> $\theta + m$

이는 이전 그레디언트의 일부를 관성으로 이해하고 있습니다.(일반적으로 $\beta$=0.9 입니다)

### 네스테로프 가속 경사

위의 모멘텀 초기화에 한가지 가정이 더해집니다.(일반적으로 맞습니다)

- 모멘텀은 언제나 최적점을 향하는 방향을 가르킵니다.

따라서 위의 식을 다음과 같이 변경합니다.

$m$ -> $\beta m- \eta \triangledown_{\theta} J(\theta+\beta m)$

$\theta$ -> $\theta + m$

즉 경사(gradient)를 모멘텀만큼 이동시킨 뒤에 측정한다는 발상입니다.

여기까지 이해했다면 아래의 파라미터들이 눈에 익을 겁니다

In [3]:
keras.optimizers.SGD(learning_rate=0.01,momentum=0.9,nesterov=True)

2025-01-13 16:22:41.429091: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-01-13 16:22:41.429109: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-01-13 16:22:41.429115: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
2025-01-13 16:22:41.429134: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-13 16:22:41.429153: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### AdaGrad



경사하강법을 생각하면 가장 가파른 방향을 찾아 변화를 줍니다.

문제는 각 변수마다의 변화량 차이가 클 경우 발생합니다.

AdaGrad는 이런 경우를 표준화를 통해 완화시키고자 하는 아이디어입니다.

아래와 같습니다.

$s \rightarrow s+ \triangledown J(\theta) \circ \triangledown J(\theta)$ ($\circ$ 은 원소별 곱셈을 나타냅니다)

$\theta \rightarrow \theta - \eta \triangledown J \oslash \sqrt{s+\epsilon}$ ($\oslash$는 원소별 나눗셈을 나타냅니다)

이를 각 $\theta_i \in \theta$ 관점에서 보면 $\eta$의 값이 감소하는 것과 같습니다.(두번쨰 단계에서)

따라서 이를 __적응형 학습률__ 이라 부릅니다.(즉 학습률 튜닝에 대한 중요도가 타 옵티마이저보다 떨어집니다)

다만 단점은 이차방정식 문제에서는 잘 작동하지만, 그외에 지나치게 일찍 멈춰 성능을 보장하기 어렵다는 것입니다.

(잘 사용하지 않지만 이후 옵티마이저의 기본이 되기에 설명했습니다)

### RMSProp

지나치게 빠르게 수렵하는 AdaGrad 의 문제를 해결하기 위해 s에 감쇠율을 적용합니다.

> <참고>

    지나치게 빠르게 수렵하는 AdaGrad의 문제는 그레디언트의 크기에 의해 발생합니다

    그레이언트가 큰 가파른 지역에서 $s$는 커집니다. 따라서 학습률을 낮추는 역할을 합니다. 이로 인해 최적점 도착전에 수렴하는 문제가 발생합니다.

위의 식을 보면 $s$는 모든 그레디언트의 원소곱을 누적하고 있음을 알 수 있습니다.

이를 비틀어 RMSProp는 최근 $s$(최근 그레디언트)에 가중치를 줍니다.

$s \rightarrow \rho s + (1-\rho) \triangledown J(\theta) \circ \triangledown J(\theta)$ ($\rho$ 는 `rho`로 읽습니다)

이는 초기 $s$는 $\rho$에 의해 지수감소하게 만듭니다 

In [286]:
keras.optimizers.RMSprop(learning_rate=0.001,rho=0.9) #파라미터 rho 가 왜 있는지 이해가 되죠?

다행히 기본값으로도 잘 작동합니다

### Adam

Adam 은 `adaptive momentum esitmation`(적응형 모멘텀 추정)을 뜻합니다.

즉 위의 AdaGrad 와 RMSProp 의 아이디어를 합칩니다.

$t$는 반복횟수입니다.

$m \rightarrow \beta_{1}m -(1-\beta_{1})\triangledown J_{\theta}$

$\hat{m} \leftarrow \frac{m}{1-\beta_{1}^{t}}$

$s \rightarrow \beta_{2}s + (1-\beta_2) \triangledown J_{\theta} \circ \triangledown J_{\theta}$

$\hat{s} \leftarrow \frac{s}{1-\beta_2^t}$

$\theta \leftarrow \theta + \eta \hat{m} \oslash \sqrt{\hat{s}+\epsilon}$

AdaGrad 의 장점을 가져오면서 당연히 __적응형 학습률__ 의 특징을 잘 계승합니다.

즉 학습률 튜닝의 중요성을 낮춰주는 좋은 방법입니다.

In [287]:
keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)

### Nadam

Adam 의 방법에 Nesterov 기법을 더한 것입니다

### AdamW

SGD 를 사용할 떄 $l_2$ 규제를 사용하는 것은 가중치의 크기를 줄이는 좋은 규제방법입니다.

이와 비슷한 방법으로 `가중치 감쇠`(모델의 가중치에 0.99와 같은 가중치를 감쇠계수를 곱하는 방법)이 있습니다.

실제 SGD를 사용할 떄 위의 가중치 감쇠 방법과 $l_2$규제가 동일한 결과를 냄이 밝혀졌지만, 

Adam과 그의 변형방법들 (AdamMax,Nadam 등)은 그렇지 않습니다.

오히려 SGD 가 만든 일반적인 모델을 만들지 못했습니다($l_2$ 규제를 적용한 결과)

따라서 가중치 감쇠방법을 적절히 사용해 위의 문제를 해결한 방법이 AdamW 입니다.

In [289]:
keras.optimizers.AdamW(learning_rate=0.001,weight_decay=0.004,beta_1=0.9,beta_2=0.999)

# weight_decay가 가중치 감쇠입니다.